In [10]:
!pip install nltk tensorflow scikit-learn
!pip install nltk tensorflow scikit-learn json

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
import json
import random
import nltk
import numpy as np
import tensorflow as tf
import string
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
import os

# Download NLTK resources
nltk.download("punkt")
nltk.download("wordnet")

# Load or Create Memory (JSON-based storage)
MEMORY_FILE = "chatbot_memory.json"

if os.path.exists(MEMORY_FILE):
    with open(MEMORY_FILE, "r") as file:
        chatbot_memory = json.load(file)
else:
    chatbot_memory = {"intents": []}

# Preprocessing
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    """
    Preprocesses text by:
    - Lowercasing
    - Removing special characters, punctuation, and extra spaces
    - Tokenizing and lemmatizing words
    """
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)])  # Lemmatization
    return text

# Load predefined intents and add learned responses
# Predefined intents with additional user identity recognition
predefined_intents = {
    "intents": [
        # Greetings
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey", "Good morning", "Good afternoon", "Howdy"],
            "responses": ["Hello! How can I assist you?", "Hey there! What’s on your mind?", "Hi! How’s your day going?"]
        },

        # Farewell
        {
            "tag": "goodbye",
            "patterns": ["Bye", "Goodbye", "See you", "Later", "Catch you later"],
            "responses": ["Goodbye! Have a great day!", "See you soon!", "Take care!"]
        },

        # Thank You
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "Appreciate it", "Thanks a lot"],
            "responses": ["You're welcome!", "No problem!", "Glad I could help!"]
        },

        # Asking About the Bot
        {
            "tag": "bot_info",
            "patterns": ["Who are you?", "What is your name?", "Are you a bot?"],
            "responses": ["I’m an AI chatbot, here to assist you!", "I’m your virtual assistant!", "I am an AI created to help with various tasks."]
        },

        # User Identity Recognition
        {
            "tag": "user_identity",
            "patterns": ["What is my name?", "Who am I?", "Do you know me?", "Do you remember me?"],
            "responses": ["I might not remember everything, but I can learn! What’s your name?", 
                          "I’d love to know your name! Can you tell me?", 
                          "I don’t seem to remember yet. What should I call you?"]
        },

        # Saving User's Name
        {
            "tag": "save_name",
            "patterns": ["My name is *", "Call me *", "I am *"],
            "responses": ["Nice to meet you, {name}!", "Great, {name}! I’ll try to remember that.", "Cool, {name}! How can I assist you today?"]
        },

        # Weather Inquiry
        {
            "tag": "weather",
            "patterns": ["What's the weather like?", "How's the weather?", "Tell me the weather"],
            "responses": ["I don’t have live weather updates yet, but you can check a weather website!", "I can’t fetch weather data right now, but you can try Google Weather!"]
        },

        # Time Inquiry
        {
            "tag": "time",
            "patterns": ["What time is it?", "Can you tell me the time?", "Current time please"],
            "responses": ["I can't check the time yet, but you can try asking your device!", "Your phone or computer should have the time displayed."]
        },

        # Learning Capability
        {
            "tag": "learning",
            "patterns": ["Can you learn?", "Do you remember things?", "How do you learn?"],
            "responses": ["I can remember new things if you teach me!", "Yes, I can store new knowledge and recall it later."]
        },

        # Hobbies & Fun
        {
            "tag": "hobbies",
            "patterns": ["What do you do for fun?", "Do you have hobbies?", "Do you like games?"],
            "responses": ["I enjoy chatting with people like you!", "I don’t have hobbies, but I can recommend fun things to do!", "Talking to people is my favorite thing!"]
        },

        # Advice & Motivation
        {
            "tag": "motivation",
            "patterns": ["I feel sad", "I need motivation", "Can you inspire me?"],
            "responses": ["You are stronger than you think! Keep pushing forward!", "Believe in yourself, and great things will happen!", "Tough times never last, but tough people do!"]
        },

        # Jokes
        {
            "tag": "joke",
            "patterns": ["Tell me a joke", "Make me laugh", "Do you know jokes?"],
            "responses": [
                "Why don’t skeletons fight each other? Because they don’t have the guts!",
                "What did one ocean say to the other? Nothing, they just waved!",
                "Why was the math book sad? Because it had too many problems!"
            ]
        },

        # Unknown
        {
            "tag": "unknown",
            "patterns": [],
            "responses": ["I’m not sure how to respond to that.", "Can you rephrase?", "I don’t know yet, but you can teach me!"]
        }
    ]
}

# Merge chatbot memory with predefined intents
for learned_intent in chatbot_memory["intents"]:
    predefined_intents["intents"].append(learned_intent)

# Prepare training data
patterns = []
labels = []
responses_dict = {}

for intent in predefined_intents["intents"]:
    for pattern in intent["patterns"]:
        processed_pattern = preprocess(pattern)
        patterns.append(processed_pattern)
        labels.append(intent["tag"])
    responses_dict[intent["tag"]] = intent["responses"]

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convert text to TF-IDF vectors
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(patterns).toarray()
y_train = np.array(labels_encoded)

# Define neural network model
model = Sequential([
    Dense(32, input_shape=(X_train.shape[1],), activation="relu"),
    Dropout(0.3),
    Dense(16, activation="relu"),
    Dense(len(set(labels)), activation="softmax")
])

# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=200, verbose=0)

# Function to add new knowledge
def learn_new_knowledge(user_input, user_response):
    # Save new knowledge
    new_intent = {
        "tag": f"learned_{len(chatbot_memory['intents'])}",
        "patterns": [user_input],
        "responses": [user_response]
    }
    chatbot_memory["intents"].append(new_intent)

    # Save to JSON file
    with open(MEMORY_FILE, "w") as file:
        json.dump(chatbot_memory, file, indent=4)

    print("Chatbot: Got it! I’ll remember this for future conversations.")

# Function to get chatbot response
def get_response(user_input):
    processed_input = preprocess(user_input)
    user_tfidf = vectorizer.transform([processed_input]).toarray()

    # Predict intent using trained model
    prediction = model.predict(user_tfidf)[0]
    predicted_index = np.argmax(prediction)
    confidence = prediction[predicted_index]

    # If confidence is high, use learned response
    if confidence > 0.5:
        tag = label_encoder.inverse_transform([predicted_index])[0]
        return random.choice(responses_dict[tag])
    else:
        return None  # Unknown response, ask user for learning

# Chat function with learning capability
def chat():
    print("Hello! I am an AI chatbot. I can also learn new things! Type 'exit' to end the conversation.")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "bye", "quit"]:
            print("Chatbot: Goodbye! Have a great day!")
            break

        response = get_response(user_input)
        
        if response:
            print(f"Chatbot: {response}")
        else:
            # Learn new information
            print("Chatbot: I don't know how to respond to that. How should I reply?")
            user_response = input("Teach me: ")
            learn_new_knowledge(user_input, user_response)

# Start chatbot
chat()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ibrez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ibrez\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Hello! I am an AI chatbot. I can also learn new things! Type 'exit' to end the conversation.


You:  hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Chatbot: Hi! How’s your day going?


You:  I'm good do you know me 


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Chatbot: I don't know how to respond to that. How should I reply?


Teach me:  how are you 


Chatbot: Got it! I’ll remember this for future conversations.


You:  who are you 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Chatbot: I’m an AI chatbot, here to assist you!
